In [8]:
import json
import re
import os
import glob
import string
from gensim.models import Word2Vec

from tqdm import tqdm

In [2]:
files=glob.glob(os.path.join('data', 'wikidata', '*', '*'))

In [3]:
texts=[]
for file in files:
    with open(file, 'r') as f:
        for line in f:
            texts.append(json.loads(line)['text'])

print(len(texts))

65041


In [4]:
def replace_strings(texts, replace):
    new_texts=[]
    
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    english_pattern=re.compile('[a-zA-Z0-9]+', flags=re.I)
    bengali_pattern=re.compile('[০-৯]+', flags=re.I)
    
    for text in tqdm(texts):
        for r in replace:
            text=text.replace(r[0], r[1])
        text=emoji_pattern.sub(r'', text)
        text=english_pattern.sub(r'', text)
        text=bengali_pattern.sub(r'', text)
        text=re.sub(r'\s+', ' ', text).strip()
        new_texts.append(text)

    return new_texts

In [5]:
def remove_punc(sentences):
    new_sentences=[]
    exclude = list(set(string.punctuation))
    exclude.extend(["’", "‘", "—"])
    for sentence in tqdm(sentences):
        s = ''.join(ch for ch in sentence if ch not in exclude)
        new_sentences.append(s)
    
    return new_sentences

In [6]:
replace=[('\u200c', ' '),
         ('\u200d', ' '),
        ('\xa0', ' '),
        ('\n', ' '),
        ('\r', ' ')]

texts=remove_punc(texts)
texts=replace_strings(texts, replace)

100%|██████████| 65041/65041 [00:21<00:00, 2958.58it/s]


In [7]:
body=[article.split('।') for article in texts]
body=[item for sublist in body for item in sublist]
body=[item.strip() for item in body if len(item.split())>1]

body=[item.split() for item in body]

print(body[:10])

[['পিটার', 'বার্জ', 'পিটার', 'জন', 'পার্নেল', 'বার্জ', 'জন্ম', 'মে', 'মৃত্যু', 'অক্টোবর', 'কুইন্সল্যান্ডের', 'ব্রিসবেনের', 'ক্যাঙ্গারু', 'পয়েন্ট', 'এলাকায়', 'জন্মগ্রহণকারী', 'বিখ্যাত', 'অস্ট্রেলীয়', 'আন্তর্জাতিক', 'ক্রিকেট', 'তারকা', 'ছিলেন'], ['থেকে', 'সময়কালে', 'অস্ট্রেলিয়া', 'ক্রিকেট', 'দলের', 'অন্যতম', 'সদস্য', 'ছিলেন', 'তিনি'], ['ঘরোয়া', 'প্রথমশ্রেণীর', 'অস্ট্রেলীয়', 'ক্রিকেটে', 'কুইন্সল্যান্ডের', 'প্রতিনিধিত্ব', 'করেছেন'], ['দলে', 'তিনি', 'মূলতঃ', 'ডানহাতি', 'ব্যাটসম্যান', 'হিসেবে', 'খেলতেন'], ['এছাড়াও', 'ডানহাতে', 'মিডিয়াম', 'বোলিংয়ে', 'পারদর্শীতা', 'দেখিয়েছেন', 'পিটার', 'বার্জ'], ['থেকে', 'সময়কালে', 'ঘরোয়া', 'অস্ট্রেলীয়', 'প্রথমশ্রেণীর', 'ক্রিকেটে', 'কুইন্সল্যান্ডের', 'পক্ষে', 'খেলেছেন', 'তিনি'], ['সমগ্র', 'খেলোয়াড়ী', 'জীবনে', 'টেস্টে', 'অংশগ্রহণ', 'করেছেন', 'পিটার', 'বার্জ'], ['ফেব্রুয়ারি', 'তারিখে', 'ইংল্যান্ডের', 'বিপক্ষে', 'টেস্ট', 'অভিষেক', 'ঘটে', 'পিটার', 'বার্জের'], ['খেলোয়াড়ী', 'জীবন', 'থেকে', 'অবসর', 'নেয়ার', 'পর', 'ম্যাচ', 'রেফারির', 'দায়িত্ব', 'প

In [10]:
model = Word2Vec(body, size=200, window=5, min_count=10)

In [11]:
print("What are the words most similar to chele")
model.wv.most_similar('ছেলে', topn=5)

What are the words most similar to chele


[('মেয়ে', 0.8783878087997437),
 ('বোন', 0.8061619997024536),
 ('ভাই', 0.7748101949691772),
 ('কন্যা', 0.7453979253768921),
 ('সন্তান', 0.7181779146194458)]

In [12]:
print("What is Father + Girl - Boy =?")
model.wv.most_similar(positive=['বাবা', 'মেয়ে'], negative=['ছেলে'], topn=5)

What is Father + Girl - Boy =?


[('মা', 0.7676048278808594),
 ('পিতা', 0.692781925201416),
 ('দাদা', 0.6826045513153076),
 ('বান্ধবী', 0.6741564869880676),
 ('মামা', 0.6636669635772705)]

In [13]:
print('Find the odd one out')
model.wv.doesnt_match("কলকাতা চেন্নাই দিল্লি রবীন্দ্রনাথ".split())

Find the odd one out


/Users/soham/.virtualenvs/upes/lib/python3.6/site-packages/gensim/models/keyedvectors.py:858: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'রবীন্দ্রনাথ'

In [16]:
model.wv.save_word2vec_format('wiki_vector_text.txt', binary=False)
model.wv.save_word2vec_format('wiki_vector_binary.txt', binary=True)